In [47]:
import importlib
import knee_dataset
importlib.reload(knee_dataset)

import os, sys

ROOT_DIR = os.path.abspath("./Mask_RCNN")
LOGS_DIR = os.path.abspath("./logs")
COCO_WEIGHTS_PATH = os.path.abspath("./weights/cocos.h5")
sys.path.append(ROOT_DIR)
from mrcnn.config import Config
from mrcnn import model as modellib, utils
import tensorflow as tf


In [48]:
def train(model, dataset_dir, config):
    """Train the model."""
    # Training dataset.
    dataset_train = knee_dataset.KneeDataset()
    dataset_train.load_knee(dataset_dir, "minitrain")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = knee_dataset.KneeDataset()
    dataset_val.load_knee(dataset_dir, "minivalid")
    dataset_val.prepare()

    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=30,
                layers='heads')


In [49]:
config = knee_dataset.KneeConfig()
print(config.DEVICE)
dataset_dir = './datasets'

with tf.device("/cpu:0"):
    model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=LOGS_DIR)

weights = 'imagenet'
if weights == "coco":
    weights_path = COCO_WEIGHTS_PATH
    # Download weights file
    if not os.path.exists(weights_path):
        utils.download_trained_weights(weights_path)
elif weights == "last":
    # Find last trained weights
    weights_path = model.find_last()
elif weights == "imagenet":
    # Start from ImageNet trained weights
    weights_path = model.get_imagenet_weights()
else:
    weights_path = weights

# Load weights
print("Loading weights ", weights_path)
if weights == "coco":
    # Exclude the last layers because they require a matching
    # number of classes
    model.load_weights(weights_path, by_name=True, exclude=[
        "mrcnn_class_logits", "mrcnn_bbox_fc",
        "mrcnn_bbox", "mrcnn_mask"])
else:
    model.load_weights(weights_path, by_name=True)

# train(model, dataset_dir, config)

/cpu:0
Loading weights  /Users/iancarvalho/.keras/models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5


In [ ]:
train(model, dataset_dir, config)

Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: /Users/iancarvalho/Documents/research/KneeSegmentation/logs/knee20190420T1939/mask_rcnn_knee_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)